# Package Import

### Standard Packages

In [125]:
import pandas as pd
import numpy as np
import sys
pd.set_option('display.max_rows', None)
import bokeh.plotting as bkp
import bokeh.io as bki
import bokeh.palettes as bkpal
from bokeh.models import Span
import time


In [2]:
bki.output_notebook()

Loading BokehJS ...

In [3]:
from bokeh.io import curdoc
from bokeh.themes import Theme

fsz = '16pt'

curdoc().theme = Theme(json={'attrs': {
    # apply defaults to Title properties
    'Title': {
        'text_font_size': fsz
    },
    # apply defaults to Axis properties
    'Axis': {
        'axis_label_text_font_size': fsz,
        'major_label_text_font_size' : fsz
    },
     # apply defaults to Legend properties
    'Legend': {
        'label_text_font_size': fsz
    }
}})

### Local Exchangeability Package

In [ ]:
import localexch

# Data Preprocessing

### Extract Relevant Columns, Simplify, Tidy

In [4]:
df = pd.read_csv('bicycle-crash-data-chapel-hill-region.csv', sep=';')
#print(df.columns)

#preprocessing data

#extract used columns
cols_used = ['BikeInjury', 'Biker Intox.', 'BikeAge', 'BikeSex', 'BikeDir', 'BikePos', 
        'DrvrVehTyp', 'LightCond', 'SpeedLimit', 'Day of Week', 'CrashHour']
df = df[cols_used]

#extract rows without missing data
ageknown = (df.BikeAge != '999') & (df.BikeAge != '70+')
lightknown = (df['LightCond'] != 'Unknown') & (df['LightCond'] != 'Other')
vehknown = df['DrvrVehTyp'] != 'Unknown'
spdknown = df['SpeedLimit'] != 'Unknown'
injknown = df['BikeInjury'] != 'Unknown Injury'
dirknown = df['BikeDir'] != 'Unknown'
posknown = df['BikePos'] != 'Unknown'
df = df[ageknown & lightknown & vehknown & spdknown & injknown & dirknown & posknown]

#convert ages to integers
df['BikeAge'] = df['BikeAge'].astype(int)

#simplify injury type
def simplify_inj(inj):
    if 'Minor' in inj:
        return 'Minor'
    elif 'Serious' in inj:
        return 'Serious'
    elif 'Killed' in inj:
        return 'Killed'
    elif 'Possible' in inj:
        return 'Possible'
    elif 'No In' in inj:
        return 'None'
    else:
        return 'NA'
df['BikeInjury'] = df['BikeInjury'].apply(simplify_inj)

#simplify light condition
df['LightCond'] = df['LightCond'].apply(lambda x : x[:4])

#simplify vehicle types
def simplify_veh(veh):
    if 'Truck' in veh or 'truck' in veh or 'Motor Home' in veh:
        return 'Truck'
    elif 'Bus' in veh:
        return 'Bus'
    elif 'EMS' in veh or 'Police' in veh:
        return 'Emergency'
    else:
        return veh
df['DrvrVehTyp'] = df['DrvrVehTyp'].apply(simplify_veh)

#simplify bike direction
def simplify_dir(dr):
    if 'With' in dr:
        return 'With'
    elif 'Not' in dr:
        return 'NA'
    else:
        return 'Against'
df['BikeDir'] = df['BikeDir'].apply(simplify_dir)

#simplify bike position
def simplify_pos(pos):
    if 'With' in pos:
        return 'With'
    elif 'Not' in dr:
        return 'NA'
    else:
        return 'Against'
df['BikeDir'] = df['BikeDir'].apply(simplify_dir)

#simplify day of week
day_map = {'Sunday': 0,
           'Monday': 1,
           'Tuesday': 2,
           'Wednesday': 3,
           'Thursday': 4,
           'Friday': 5,
           'Saturday': 6}
df['Day of Week'] = df['Day of Week'].apply(lambda x : day_map[x])

#simplify speed limit
def simplify_spd(spd):
    return spd.strip().split(' ')[2]
df['SpeedLimit'] = df['SpeedLimit'].apply(simplify_spd)
df['SpeedLimit'] = df['SpeedLimit'].astype(int)

print('Single Dataframe Version:')
print(df.head())

print('')
print('')
print('Split Covariate/Outcome:')

#create a version of the data with covariates and observations separate
Y = np.array(df['BikeInjury'])
X = np.array(df.drop(['BikeInjury'], axis=1))

print('X:')
print(X)
print('Y:')
print(Y)

Single Dataframe Version:
  BikeInjury Biker Intox.  BikeAge BikeSex  BikeDir  \
0       None           No       12    Male     With   
2   Possible           No       36    Male     With   
3   Possible           No       63    Male     With   
4   Possible           No       14    Male     With   
5       None           No       57    Male  Against   

                      BikePos     DrvrVehTyp LightCond  SpeedLimit  \
0                 Travel Lane  Passenger Car      Dayl          35   
2  Bike Lane / Paved Shoulder  Passenger Car      Dayl          45   
3                 Travel Lane  Sport Utility      Dayl          35   
4                 Travel Lane  Passenger Car      Dayl          35   
5                 Non-Roadway         Pickup      Dayl          15   

   Day of Week  CrashHour  
0            6         18  
2            4         17  
3            3         18  
4            3         15  
5            6         13  


Split Covariate/Outcome:
X:
[['No' 12 'Male' ... 35 

# Local Permutation Test

Does biker intoxication affect probability of severe injury?

The null hypothesis is that Biker Intoxication doesn't affect the probability of severe injury. Therefore we assume the observations are exchangeable w.r.t. that covariate, while the others are dealt with via the premetric:

### Premetric

In [79]:
#X columns: ['Biker Intox.', 'BikeAge', 'BikeSex', 'BikeDir', 'BikePos', 
#        'DrvrVehTyp', 'LightCond', 'SpeedLimit', 'Day of Week', 'CrashHour']

def premetric(x1, x2, weight):
    cat_idcs = [2,3,4,5,6]
    if (x1[cat_idcs] != x2[cat_idcs]).any():
        return 1
    else:
        return min(1, 
                  weight*(
                  np.fabs(x1[1]-x2[1]) + 
                  np.fabs(x1[7]-x2[7]) +
                  min(np.fabs(x1[8]-x2[8]), 7 - np.fabs(x1[8]-x2[8])) + 
                  min(np.fabs(x1[9]-x2[9]), 24 - np.fabs(x1[9]-x2[9]))
                  ))           

### Test Statistic

In [80]:
def test_stat(X, Y):
    intox = (X[:, 0] == 'Yes')
    severe = (Y == 'Serious') | (Y == 'Killed') 
    return (severe & intox).sum()/intox.sum() - (severe & ~intox).sum()/(~intox).sum()

### Pair Permutation Subgroup Constructor

In [81]:
def construct_pairs(X, d, match_coln):
    #ensure the match column has exactly two values
    match_vals = np.unique(X[:, 0])
    if match_vals.shape[0] != 2:
        print('Unable to construct pair matching on column ' + str(match_coln))
        print('Number of unique values != 2: ' +str(match_vals.shape[0]))
    idcs0 = np.where(X[:, match_coln] == match_vals[0])[0]
    idcs1 = np.where(X[:, match_coln] == match_vals[1])[0]
    X0 = X[X[:, match_coln] == match_vals[0]]
    X1 = X[X[:, match_coln] == match_vals[1]]
    
    dists = np.zeros((X0.shape[0], X1.shape[0]))
    for i in range(X0.shape[0]):
        if i % 1000 == 0:
            sys.stdout.write('row ' + str(i+1)+'/'+str(X0.shape[0])+'              \r')
            sys.stdout.flush()
        for j in range(X1.shape[0]):
            dists[i,j] = d(X0[i,:], X1[j,:])
    sys.stdout.write('\n')
    sys.stdout.flush()
    pairs = []
    pair_dists = []
    while dists.min() < np.inf:
        am = dists.argmin()
        row = am // dists.shape[1]
        col = am % dists.shape[1]
        pairs.append((idcs0[row], idcs1[col]))
        pair_dists.append(dists[row, col])
        dists[row, :] = np.inf
        dists[:, col] = np.inf
        
    return pairs, pair_dists

### Paired Permutation Test

In [82]:
def pair_test(X, Y, S, pairs, pair_dists, N_samples, alpha):
    #take pairs until their local exchangeability penalty would exceed alpha/2
    num_pairs = (np.cumsum(pair_dists) <= alpha/2).sum()
    
    if num_pairs == 0:
        return False, 0, S(X, Y), S(X, Y)*np.ones(N_samples) #all "permutations" are the same, cannot reject null
    
    #compute the local exchangeability penalty
    penalty = np.cumsum(pair_dists)[num_pairs-1]
    thresh = 1 - alpha + penalty
    
    #run N_samples of pair swaps 
    S_vals = np.zeros(N_samples)
    for n in range(N_samples):
        if n % 1000 == 0:
            sys.stdout.write('sample ' + str(n+1)+'/'+str(N_samples)+'              \r')
            sys.stdout.flush()
        Yc = Y.copy()
        for p in range(num_pairs):
            if np.random.rand() <= 0.5:
                tmp = Yc[pairs[p][0]]
                Yc[pairs[p][0]] = Yc[pairs[p][1]]
                Yc[pairs[p][1]] = tmp
        S_vals[n] = S(X, Yc)
    sys.stdout.write('\n')
    sys.stdout.flush()
    
    reject_null = (S(X, Y) > S_vals).sum()/N_samples > thresh
    
    return reject_null, num_pairs, S(X, Y), S_vals

### Run the Test

In [89]:
weights = np.logspace(-7, 0, 8)
match_col = 0
alpha = 0.05
N_samples = 100000

S_vals = np.zeros((weights.shape[0], N_samples))
S = 0
n_pairs = np.zeros(weights.shape[0])
rejections = np.zeros(weights.shape[0], dtype=np.bool)
for n in range(weights.shape[0]):
    print('Weight iter ' + str(n+1)+'/'+str(weights.shape[0]))
    #construct the list of matched pairs
    pairs, pair_dists = construct_pairs(X, lambda x1, x2 : premetric(x1, x2, weights[n]), match_col)
    
    print(pairs)
    print(pair_dists)
    #run the permutation test
    rejections[n], n_pairs[n], S, S_vals[n,:] = pair_test(X, Y, test_stat, pairs, pair_dists, N_samples, alpha)   


Weight iter 1/14
row 8001/8234              
[(0, 343), (1, 831), (2, 274), (3, 426), (4, 2242), (5, 487), (6, 402), (7, 1545), (8, 151), (11, 615), (13, 169), (14, 520), (17, 1633), (18, 3050), (19, 1707), (20, 710), (21, 268), (22, 1736), (23, 887), (24, 647), (25, 716), (26, 279), (27, 1939), (28, 5557), (29, 287), (30, 1612), (31, 881), (32, 480), (33, 5823), (35, 1433), (36, 1158), (37, 198), (38, 768), (39, 1580), (40, 1851), (41, 3839), (42, 295), (43, 2385), (44, 2056), (45, 3885), (46, 5849), (47, 5704), (48, 525), (49, 1093), (50, 987), (51, 696), (53, 1208), (54, 8324), (55, 2658), (56, 1484), (57, 6439), (58, 5060), (59, 2985), (62, 1797), (64, 1945), (65, 7937), (69, 2068), (70, 1238), (71, 2098), (73, 3840), (74, 1387), (76, 2141), (77, 2218), (79, 945), (80, 2387), (81, 2405), (82, 1427), (86, 2446), (87, 2028), (89, 1536), (90, 4117), (91, 302), (92, 2378), (93, 1472), (94, 2478), (95, 2584), (96, 3572), (97, 4504), (98, 2312), (99, 2847), (100, 2895), (101, 1131), (103

sample 99001/100000              
Weight iter 2/14
row 8001/8234              
[(14, 3366), (820, 3901), (1865, 3638), (3084, 4563), (7273, 1545), (7342, 8370), (7362, 3647), (7456, 5582), (8399, 7636), (8704, 7571), (29, 2788), (100, 7742), (179, 6879), (251, 8467), (252, 343), (488, 3487), (494, 3941), (504, 2185), (740, 5332), (806, 8033), (1132, 4345), (1139, 1115), (1206, 1883), (1323, 936), (1404, 7043), (1787, 787), (1836, 3654), (1924, 6658), (2146, 4914), (2322, 4474), (2349, 528), (2369, 295), (2731, 6350), (2770, 3396), (3011, 753), (3046, 8006), (3202, 7124), (3315, 4805), (3625, 8244), (3910, 520), (3927, 6346), (4030, 5919), (4053, 8157), (4460, 7492), (4526, 2068), (4561, 3176), (4650, 5550), (4802, 5245), (4842, 5213), (4972, 8771), (5134, 8255), (5269, 5931), (5490, 2895), (5723, 7471), (5811, 6296), (5900, 2584), (6086, 2160), (6276, 3855), (6628, 6327), (6653, 5839), (6808, 1387), (6825, 1450), (6918, 7537), (6919, 706), (6939, 5668), (7226, 7730), (7296, 2985), (737

sample 99001/100000              
Weight iter 3/14
row 8001/8234              
[(14, 3366), (820, 3901), (1865, 3638), (3084, 4563), (7273, 1545), (7342, 8370), (7362, 3647), (7456, 5582), (8399, 7636), (8704, 7571), (29, 2788), (100, 7742), (179, 6879), (251, 8467), (252, 343), (488, 3487), (494, 3941), (504, 2185), (740, 5332), (806, 8033), (1132, 4345), (1139, 1115), (1206, 1883), (1323, 936), (1404, 7043), (1787, 787), (1836, 3654), (1924, 6658), (2146, 4914), (2322, 4474), (2349, 528), (2369, 295), (2731, 6350), (2770, 3396), (3011, 753), (3046, 8006), (3202, 7124), (3315, 4805), (3625, 8244), (3910, 520), (3927, 6346), (4030, 5919), (4053, 8157), (4460, 7492), (4526, 2068), (4561, 3176), (4650, 5550), (4802, 5245), (4842, 5213), (4972, 8771), (5134, 8255), (5269, 5931), (5490, 2895), (5723, 7471), (5811, 6296), (5900, 2584), (6086, 2160), (6276, 3855), (6628, 6327), (6653, 5839), (6808, 1387), (6825, 1450), (6918, 7537), (6919, 706), (6939, 5668), (7226, 7730), (7296, 2985), (737

sample 99001/100000              
Weight iter 4/14
row 8001/8234              
[(14, 3366), (820, 3901), (1865, 3638), (3084, 4563), (7273, 1545), (7342, 8370), (7362, 3647), (7456, 5582), (8399, 7636), (8704, 7571), (29, 2788), (100, 7742), (179, 6879), (251, 8467), (252, 343), (488, 3487), (494, 3941), (504, 2185), (740, 5332), (806, 8033), (1132, 4345), (1139, 1115), (1206, 1883), (1323, 936), (1404, 7043), (1787, 787), (1836, 3654), (1924, 6658), (2146, 4914), (2322, 4474), (2349, 528), (2369, 295), (2731, 6350), (2770, 3396), (3011, 753), (3046, 8006), (3202, 7124), (3315, 4805), (3625, 8244), (3910, 520), (3927, 6346), (4030, 5919), (4053, 8157), (4460, 7492), (4526, 2068), (4561, 3176), (4650, 5550), (4802, 5245), (4842, 5213), (4972, 8771), (5134, 8255), (5269, 5931), (5490, 2895), (5723, 7471), (5811, 6296), (5900, 2584), (6086, 2160), (6276, 3855), (6628, 6327), (6653, 5839), (6808, 1387), (6825, 1450), (6918, 7537), (6919, 706), (6939, 5668), (7226, 7730), (7296, 2985), (737

sample 99001/100000              
Weight iter 5/14
row 8001/8234              
[(14, 3366), (820, 3901), (1865, 3638), (3084, 4563), (7273, 1545), (7342, 8370), (7362, 3647), (7456, 5582), (8399, 7636), (8704, 7571), (29, 2788), (100, 7742), (179, 6879), (251, 8467), (252, 343), (488, 3487), (494, 3941), (504, 2185), (740, 5332), (806, 8033), (1132, 4345), (1139, 1115), (1206, 1883), (1323, 936), (1404, 7043), (1787, 787), (1836, 3654), (1924, 6658), (2146, 4914), (2322, 4474), (2349, 528), (2369, 295), (2731, 6350), (2770, 3396), (3011, 753), (3046, 8006), (3202, 7124), (3315, 4805), (3625, 8244), (3910, 520), (3927, 6346), (4030, 5919), (4053, 8157), (4460, 7492), (4526, 2068), (4561, 3176), (4650, 5550), (4802, 5245), (4842, 5213), (4972, 8771), (5134, 8255), (5269, 5931), (5490, 2895), (5723, 7471), (5811, 6296), (5900, 2584), (6086, 2160), (6276, 3855), (6628, 6327), (6653, 5839), (6808, 1387), (6825, 1450), (6918, 7537), (6919, 706), (6939, 5668), (7226, 7730), (7296, 2985), (737

sample 99001/100000              
Weight iter 6/14
row 8001/8234              
[(14, 3366), (820, 3901), (1865, 3638), (3084, 4563), (7273, 1545), (7342, 8370), (7362, 3647), (7456, 5582), (8399, 7636), (8704, 7571), (29, 2788), (100, 7742), (179, 6879), (251, 8467), (252, 343), (488, 3487), (494, 3941), (504, 2185), (740, 5332), (806, 8033), (1132, 4345), (1139, 1115), (1206, 1883), (1323, 936), (1404, 7043), (1787, 787), (1836, 3654), (1924, 6658), (2146, 4914), (2322, 4474), (2349, 528), (2369, 295), (2731, 6350), (2770, 3396), (3011, 753), (3046, 8006), (3202, 7124), (3315, 4805), (3625, 8244), (3910, 520), (3927, 6346), (4030, 5919), (4053, 8157), (4460, 7492), (4526, 2068), (4561, 3176), (4650, 5550), (4802, 5245), (4842, 5213), (4972, 8771), (5134, 8255), (5269, 5931), (5490, 2895), (5723, 7471), (5811, 6296), (5900, 2584), (6086, 2160), (6276, 3855), (6628, 6327), (6653, 5839), (6808, 1387), (6825, 1450), (6918, 7537), (6919, 706), (6939, 5668), (7226, 7730), (7296, 2985), (737

KeyboardInterrupt: 

### Visualize the Result

In [88]:
fig = bkp.figure(x_range =(0, .15), x_axis_label='Statistic Value', y_axis_label='Density')
vline = Span(location=S, dimension='height', line_color='black', line_width=3)
fig.renderers.append(vline)

for n in range(weights.shape[0]):
    hist, edges = np.histogram(S_vals[n,:], bins=np.linspace(0, 0.15, 90))
    fig.quad(top=hist/N_samples, bottom=0, left=edges[:-1], right=edges[1:],
               fill_color=bkpal.Category20[20][n], line_color="white", alpha=0.4, legend='Weight = ' + str(weights[n]))
bkp.show(fig)

fig = bkp.figure(x_axis_label='Weight', y_axis_label='# Pairs', x_axis_type='log')
fig.line(weights, n_pairs, line_width=3)
bkp.show(fig)

In [70]:
print(S_vals.mean(axis=1))

[0.07331641 0.08261997 0.08266431 0.08266865 0.08268607 0.11575285
 0.12167969 0.12339066 0.12253048]


In [85]:
print(n_pairs)

[628. 628. 628. 628. 556. 167.  34.  12.  10.  10.]


### Run the test

In [ ]:
localexch.test(X, Y, test_stat, group_sampler, premetric, n_samples)

### Visualization

# Data Visualization

In [108]:
#obtain rows of df for intoxicated biker / severe outcomes
intox = (df['Biker Intox.'] == 'Yes')
severe = (df['BikeInjury'] == 'Serious') | (df['BikeInjury'] == 'Killed') 

indices = [ (~severe & ~intox), (~severe & intox), (severe & ~intox),  (severe & intox) ]
colors = [bkpal.Category10[10][0], bkpal.Category10[10][0], bkpal.Category10[10][1], bkpal.Category10[10][1]]
alphas = [0.3, 0.7, 0.3, 0.7]
labels=['Not Severe, Not Intoxicated', 'Not Severe, Intoxicated', 'Severe, Not Intoxicated', 'Severe, Intoxicated']

coln = df.DrvrVehTyp.unique()
vals = np.zeros(coln.shape[0])
totalcts = df.DrvrVehTyp.value_counts()[coln]
fig = bkp.figure(width=800, height=400, y_range=coln,
                x_axis_label='Fraction')
print(totalcts)
for i in range(4):
    cts = df.DrvrVehTyp[indices[i]].value_counts()[coln]
    cts[np.isnan(cts)] = 0
    fig.hbar(y = coln, left = vals/totalcts, right=(vals+cts)/totalcts,
                 fill_color=colors[i], line_color='black', line_width=0.5, height=.7, fill_alpha=alphas[i])
    vals = vals + cts
bkp.show(fig)


coln = np.sort(df.SpeedLimit.unique())
vals = np.zeros(coln.shape[0])
totalcts = df.SpeedLimit.value_counts()[coln]
fig = bkp.figure(width=600, height=250, y_range=[str(x) for x in coln],
                x_axis_label='Fraction',
                y_axis_label='Speed Limit (MPH)')
for i in range(4):
    cts = df.SpeedLimit[indices[i]].value_counts()[coln]
    cts[np.isnan(cts)] = 0
    fig.hbar(y = [str(x) for x in coln], left = vals/totalcts, right=(vals+cts)/totalcts,
                 fill_color=colors[i], line_color='black', line_width=0.5, height=.7, fill_alpha=alphas[i])
    vals = vals + cts
bkp.show(fig)


coln = np.sort(df.BikeSex.unique())
vals = np.zeros(coln.shape[0])
totalcts = df.BikeSex.value_counts()[coln]
fig = bkp.figure(width=600, height=250, y_range=[str(x) for x in coln],
                x_axis_label='Fraction',
                y_axis_label='Speed Limit (MPH)')
for i in range(4):
    cts = df.BikeSex[indices[i]].value_counts()[coln]
    cts[np.isnan(cts)] = 0
    fig.hbar(y = [str(x) for x in coln], left = vals/totalcts, right=(vals+cts)/totalcts,
                 fill_color=colors[i], line_color='black', line_width=0.5, height=.7, fill_alpha=alphas[i])
    vals = vals + cts
bkp.show(fig)

coln = np.array(['Dawn', 'Dayl', 'Dusk', 'Dark'])
vals = np.zeros(coln.shape[0])
totalcts = df.LightCond.value_counts()[coln]
fig = bkp.figure(width=600, height=200, y_range=[str(x) for x in coln],
                x_axis_label='Fraction')
for i in range(4):
    cts = df.LightCond[indices[i]].value_counts()[coln]
    cts[np.isnan(cts)] = 0
    fig.hbar(y = [str(x) for x in coln], left = vals/totalcts, right=(vals+cts)/totalcts,
                 fill_color=colors[i], line_color='black', line_width=0.5, height=.7, fill_alpha=alphas[i])
    vals = vals + cts
bkp.show(fig)


coln = np.sort(df.BikeAge.unique())
vals = np.zeros(coln.shape[0])
totalcts = df.BikeAge.value_counts()[coln]
fig = bkp.figure(width=600, height=2000, y_range=[str(x) for x in coln],
                x_axis_label='Fraction',
                y_axis_label='Speed Limit (MPH)')
for i in range(4):
    cts = df.BikeAge[indices[i]].value_counts()[coln]
    cts[np.isnan(cts)] = 0
    fig.hbar(y = [str(x) for x in coln], left = vals/totalcts, right=(vals+cts)/totalcts,
                 fill_color=colors[i], line_color='black', line_width=0.5, height=.7, fill_alpha=alphas[i])
    vals = vals + cts
bkp.show(fig)


coln = np.sort(df.LightCond.unique())
vals = np.zeros(coln.shape[0])
totalcts = df.LightCond.value_counts()
fig = bkp.figure(width=600, height=400, y_range=[str(x) for x in coln],
                x_axis_label='Fraction', x_range=(0, 5))
for i in range(4):
    cts = df.LightCond[indices[i]].value_counts()[coln]
    cts[np.isnan(cts)] = 0
    fig.hbar(y = [str(x) for x in coln], left = vals/totalcts, right=(vals+cts)/totalcts,
                 fill_color=colors[i], line_color='black', line_width=0.5, height=.7, fill_alpha=alphas[i],
                legend=labels[i])
    vals = vals + cts
bkp.show(fig)

Passenger Car           5051
Sport Utility           1623
Pickup                  1271
Truck                    261
Van                      466
Motorcycle                50
Bus                       50
Emergency                 50
Tractor/Semi-Trailer      24
Taxicab                   17
Moped                      6
Name: DrvrVehTyp, dtype: int64


/home/trevor/.local/lib/python3.6/site-packages/pandas/core/series.py:1139: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


# 

In [109]:
unique_bin_cts = df.groupby(['BikeAge','BikeSex','Biker Intox.', 'BikeDir', 'BikePos', 'Day of Week', 'CrashHour', 'DrvrVehTyp','LightCond','SpeedLimit']).count()
#print(unique_bin_cts)
#unique_bin_cts = df.groupby(['BikeSex','Biker Intox.', 'BikeDir', 'BikePos', 'DrvrVehTyp','LightCond','SpeedLimit']).count()


fig = bkp.figure(x_axis_type='log')
hist, edges = np.histogram(unique_bin_cts['BikeInjury'], bins=np.logspace(0, 3, 12))
fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color=bkpal.Category10[10][0], line_color="white", alpha=0.4, legend='Frac Severe, Not Intox')
bkp.show(fig)

# Local Estimation

### Premetric

In [141]:
#covariate = 'CrashHour'
def premetric(x1, x2, weight):
    return min(1, weight*min(np.fabs(x1-x2), 24 - np.fabs(x1-x2)))

In [142]:
def empirical_msr(x, X, premetric):
    b = np.zeros(X.shape[0])
    for n in range(X.shape[0]):
        b[n] = premetric(x, X[n])
    idcs = np.argsort(b)
    c1 = 1. + 2*np.cumsum(b[idcs])
    c2 = 2*(1.+np.arange(b.shape[0]))*b[idcs]
    M = (c1 > c2).sum()
    mu = (1.+2*b[idcs][:M].sum())/M
    w = np.maximum(-2*b + mu, 0)
    return w

What is the probability of severe outcome as a function of biker age / intoxication state

In [144]:
weights = np.logspace(-7, 0, 8)
times = np.linspace(0, 23, 50)
ests = np.zeros((weights.shape[0], times.shape[0]))
Ybin = np.zeros(Y.shape[0])
severe = (Y == 'Serious') | (Y == 'Killed') 
Ybin[severe] = 1
for n in range(weights.shape[0]):
    print('weight iteration ' + str(n+1)+'/'+str(weights.shape[0]))
    for j in range(times.shape[0]):
        emp_msr_wts = empirical_msr(times[j], X[:, 9], lambda x1, x2 : premetric(x1, x2, weights[n]))
        ests[n, j] = (emp_msr_wts*Ybin).sum()

weight iteration 1/8
weight iteration 2/8
weight iteration 3/8
weight iteration 4/8
weight iteration 5/8
weight iteration 6/8
weight iteration 7/8
weight iteration 8/8


In [128]:
#Plot bars for fraction of severe outcomes at each time
fig = bkp.figure(x_range=(0,25))
for n in range(weights.shape[0]):
    fig.line(times, ests[n,:], line_width=3)
    fig.quad(top=[(Ybin[X[:,9] == t].sum()/(X[:,9]==t).sum()) for t in range(24)], bottom=0, 
             left=np.arange(24)-.5, right=np.arange(24)+.5, line_color='white', alpha=0.1)
bkp.show(fig)

In [147]:
n_data = np.logspace(0, 4, 10).astype(int)
cputs = np.zeros(n_data.shape[0])
wt = 0.001
idcs = np.arange(X.shape[0])
np.random.shuffle(idcs)
for n in range(n_data.shape[0]):
    print('ndata iter ' + str(n+1)+'/'+str(n_data.shape[0]))
    t0 = time.perf_counter()
    times = np.linspace(0, 23, 50)
    ests = np.zeros(times.shape[0])
    Ybin = np.zeros(Y.shape[0])
    severe = (Y == 'Serious') | (Y == 'Killed') 
    Ybin[severe] = 1
    for j in range(times.shape[0]):
        emp_msr_wts = empirical_msr(times[j], X[idcs[:n_data[n]], 9], lambda x1, x2 : premetric(x1, x2, wt))
        ests[j] = (emp_msr_wts*Ybin[idcs[:n_data[n]]]).sum()
    tf = time.perf_counter()
    cputs[n] = tf-t0

ndata iter 1/10
ndata iter 2/10
ndata iter 3/10
ndata iter 4/10
ndata iter 5/10
ndata iter 6/10
ndata iter 7/10
ndata iter 8/10
ndata iter 9/10
ndata iter 10/10


In [149]:
#Plot bars for fraction of severe outcomes at each time
fig = bkp.figure()
fig.line(n_data, cputs, line_width=3)
bkp.show(fig)

In [ ]:

intox = (df['Biker Intox.'] == 'Yes')
severe = (df['BikeInjury'] == 'Serious') | (df['BikeInjury'] == 'Killed')

print((severe & intox).sum() / ((severe & intox).sum() +(~severe & intox).sum() ))
print((severe & ~intox).sum() / ((severe & ~intox).sum() +(~severe & ~intox).sum() ))


fig = bkp.figure()

hist1, edges = np.histogram(df['BikeAge'][~severe & ~intox], bins=np.arange(0, 100, 1))
hist2, edges = np.histogram(df['BikeAge'][severe & ~intox], bins=np.arange(0, 100, 1))
fig.quad(top=hist2/(hist1+hist2), bottom=0, left=edges[:-1], right=edges[1:],
           fill_color=bkpal.Category10[10][0], line_color="white", alpha=0.4, legend='Frac Severe, Not Intox')

hist1, edges = np.histogram(df['BikeAge'][~severe & intox], bins=np.arange(0, 100, 1))
hist2, edges = np.histogram(df['BikeAge'][severe & intox], bins=np.arange(0, 100, 1))
fig.quad(top=hist2/(hist1+hist2), bottom=0, left=edges[:-1], right=edges[1:],
           fill_color=bkpal.Category10[10][1], line_color="white", alpha=0.4, legend='Frac Severe, Intox')


bkp.show(fig)

In [ ]:
fig = bkp.figure()
hist, edges = np.histogram(df['BikeAge'], bins=np.arange(0, 100, 1))
fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color=bkpal.Category10[10][0], line_color="white", alpha=0.4)
bkp.show(fig)

In [ ]:
print(df.BikeInjury.unique())



In [ ]:
intoxrows = (df['Biker Intox.'] == 'Yes')
intox = df[cols_used][intoxrows]
nintox = df[cols_used][~intoxrows]
print(intox.shape)
print(nintox.shape)
injuries = (intox['BikeInjury'] == 'A: Suspected Serious Injury') | (intox['BikeInjury'] == 'K: Killed')
print(injuries.sum())
injuries = (nintox['BikeInjury'] == 'A: Suspected Serious Injury') | (nintox['BikeInjury'] == 'K: Killed')
print(injuries.sum())

In [ ]:
139/796

In [ ]:
688/10470